# Find all the types of documents posible in supercias
### We need to define how many types documents and their frequency in supercias

In [1]:
packages = ['pymongo', 'gridfs', 'pandas', 'pymupdf', 'pytesseract', 'pdf2image', 'openai', 'python-dotenv']
for package in packages:
    try:
        __import__(package)
    except ImportError:
        !pip install {package}

import os
from pymongo import MongoClient
import pandas as pd
from functions.pdf_operations import PDFOperations
from functions.prompt_openai import query_openai
# endpoint = '10.0.10.5:27017' # use vpn
endpoint = '192.168.1.10:27017'  # use lan
#endpoint = '192.168.229.55:27017' # use house wifi
database = 'supercias'
collection = 'clean_companies'
# Connect to MongoDB
db = MongoClient('mongodb://'+endpoint)[database]
# get collection
collection = db[collection]
# where the pdfs will be stored 
pdfs_path = '../storage/pdfs/'
# pdf operator
pdf_op = PDFOperations(db, 'companies', pdfs_path)
#find pdf file
companies_cursor = collection.find()


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


Let seperate the docuemt by the type they can belong to: Generales, Economicos  and Judiciales

In [2]:
# now lets get all the documents for every company
companies = collection.find()
Documentos_generales = []
Documentos_economicos = []
Documentos_juridicos = []

for company in companies:
    # get the Documentos online
    documentos = company['Documentos']
    for documento in documentos['Generales']:
        Documentos_generales.append(documento)
    for documento in documentos['Economicos']:
        Documentos_economicos.append(documento)
    for documento in documentos['Juridicos']:
        Documentos_juridicos.append(documento)

Get all of the difrent kind of documentos generales into a df from 2010 and on

In [3]:
def proces_documents(documentos, label_fields):
    docs = []
    for documento in documentos:
        fields = documento.split('.')[0].split('_')
        new_doc = {}
        for index, field in enumerate(fields):
            new_doc[label_fields[index]] = field
        # add filename at the end
        new_doc['filename'] = documento
        # add new doc
        docs.append(new_doc)
    return docs

doc_generales_fields = ['ruc', 'tipo_doc', 'documento', 'fecha', 'nombre', 'cargo', 'other']
docs = proces_documents(Documentos_generales, doc_generales_fields)
df_generales = pd.DataFrame(docs)
df_generales['fecha'] = pd.to_datetime(df_generales['fecha'])
# get all of the docuemntos from 2010 and on
df_generales = df_generales[df_generales['fecha'] > '2010-01-01']
df_generales.head()

,ruc,tipo_doc,documento,fecha,filename,nombre,cargo
0,0791841030001,DocumentosGenerales,Formulario de actualización de datos,2022-08-04,0791841030001_DocumentosGenerales_Formulario d...,NaN,NaN
1,0791841030001,DocumentosGenerales,Formulario de actualización de datos,2024-04-02,0791841030001_DocumentosGenerales_Formulario d...,NaN,NaN
2,0791841030001,DocumentosGenerales,Nombramiento,2023-01-26,0791841030001_DocumentosGenerales_Nombramiento...,FAJARDO TINOCO ISAURO DE JESUS,PRESIDENTE
3,0791841030001,DocumentosGenerales,Nombramiento SAS,2022-08-02,0791841030001_DocumentosGenerales_Nombramiento...,FAJARDO FAJARDO LESLIE FABIOLA,GERENTE GENERAL
4,0791841030001,DocumentosGenerales,Formulario de actualización de datos,2023-08-07,0791841030001_DocumentosGenerales_Formulario d...,NaN,NaN


In [4]:
# 0791841030001_DocumentosGenerales_Formulario de actualización de datos_2022-08-04.pdf
# get all the ditribution of the documents
print(df_generales['documento'].value_counts())

documento
Oficio Nombramiento Administradores                        106791
Formulario de actualización de datos                        86756
Oficio Transferencia Acciones                               48987
Razón de Inscripción registrada                             39292
Nombramiento                                                38436
Formulario de Información de la Sociedad Extranjera         26174
Certificado de Existencia Legal                             26118
Poder otorgado por la Sociedad Extranjera                   25864
Declaración Juramentada  Nómina de Socios o Accionistas     14601
Solicitud de acceso y declaración de responsabilidad        13575
Carta de autorización                                        7693
Escritura Cesión Participación                               6622
Notificación general                                         2696
Razón de inscripción del nombramiento                        2315
Renuncia                                                     2174


We define the schema to be used by to scrap the Oficio Nombramiento Administradores

In [5]:
# extracted structure for 'Oficio Nombramiento Administradores'
from pydantic import BaseModel
from typing import List, Optional

class PreviousHolder(BaseModel):
    name: str
    designated_date: str
    id_number: str
    id_type: str

class NewHolder(BaseModel):
    name: str
    designated_date: str
    id_number: str
    id_type: str

class Appointment(BaseModel):
    position: str
    term_years: str
    appointment_date: str
    previous_holder: Optional[PreviousHolder]
    new_holder: Optional[NewHolder]

class Company(BaseModel):
    ruc: str
    name: str
    region: str
    date: str
    address: str
    phone: str
    notary: str
    lawyer: str

class OficioNombramientoAdministradores(BaseModel):
    company: Company
    appointment: Appointment

Now we evtract the text from every possible Oficio Nombramiento Administradores

We need to define what data we want to extact from every Oficio Nombramiento Administradores

In [7]:
import concurrent.futures
import threading
import logging
import traceback

# Configure the logger
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(threadName)s - %(message)s')

# Create a global lock for printing (if needed)
print_lock = threading.Lock()

# create a collection for Oficio Nombramiento Administradores un cmongoDB
collection = db['Oficio_Nombramiento_Administradores']
# get all the documents which have the document 'Oficio Nombramiento Administradores' and have the date containes '2023'
df_oficio_nombramiento_administradores = df_generales[(df_generales['documento'] == 'Oficio Nombramiento Administradores')]


def extract_data_from_pdf(row, collection, pdf_op, schema):
    # check if the file exists in the collection
    filename = row['filename']
    if collection.find_one({'filename': filename}):
        #print(f'{filename} already exists in the collection')
        return
    print(f'Extracting data from {filename}')
    text = pdf_op.extract_text(filename)
    if( text == None):
        print(f'Error extracting text from {filename}')
        return
    # get the size of the text
    tokens = len(text.split())
    if tokens < 100:
        print(f'Text size: {tokens}')
        print(f'Not enough text in {filename}')
        return
    elif tokens > 16385:
        print(f'Text size: {tokens}')
        print(f'Too much text in {filename}')
        return
    # query open AI
    response = query_openai("extract the following data in the json format, with any N/A, from the following text: " + text, schema=schema)
    # convert the response to a dictionary
    response = response.dict()
    response['ruc'] = row['ruc']
    response['fecha'] = row['fecha']
    response['nombre'] = row['nombre']
    response['cargo'] = row['cargo']
    response['filename'] = filename
    print(response)
    # add to the collection
    collection.insert_one(response)
    # return response 
    print('-----------------------------------')
    return response


Run Sequentially

In [25]:
# test with the filename '1790721450001_DocumentosGenerales_Oficio Nombramiento Administradores_2020-09-04_ALVERNIA DE CHACON IMELDA_PRESIDENTE.pdf'
#filename = '1790721450001_DocumentosGenerales_Oficio Nombramiento Administradores_2020-09-04_ALVERNIA DE CHACON IMELDA_PRESIDENTE.pdf'
#df_oficio_nombramiento_administradores = df_oficio_nombramiento_administradores[df_oficio_nombramiento_administradores['filename'] == filename]
# test the first values of the df between 100 and 200
for _, row in df_oficio_nombramiento_administradores.iterrows():
    res = extract_data_from_pdf(row, collection, pdf_op, OficioNombramientoAdministradores)

Extracting data from 1790721450001_DocumentosGenerales_Oficio Nombramiento Administradores_2020-09-04_ALVERNIA DE CHACON IMELDA_PRESIDENTE.pdf


2024-10-13 21:10:22,265 - MainThread - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'company': {'ruc': 'N/A', 'name': 'LABORATORIOS CHALVER DEL ECUADOR CIA LTDA.', 'region': 'Quito', 'date': '30 de julio de 2020', 'address': 'N/A', 'phone': 'N/A', 'notary': 'Doctor Guillermo Salen Rivas, Notario Octavo del cantón Quito', 'lawyer': 'N/A'}, 'appointment': {'position': 'PRESIDENTE', 'term_years': '5', 'appointment_date': '30 de julio de 2020', 'previous_holder': None, 'new_holder': {'name': 'Imelda Alvernia de Chacón', 'designated_date': '30 de julio de 2020', 'id_number': 'AM794539', 'id_type': 'Pasaporte'}}, 'ruc': '1790721450001', 'fecha': Timestamp('2020-09-04 00:00:00'), 'nombre': 'ALVERNIA DE CHACON IMELDA', 'cargo': 'PRESIDENTE', 'filename': '1790721450001_DocumentosGenerales_Oficio Nombramiento Administradores_2020-09-04_ALVERNIA DE CHACON IMELDA_PRESIDENTE.pdf'}


Run multithreaded

In [ ]:
# Multithreading implementation
def process_pdf_data_in_parallel(df, collection, max_workers=5):
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit tasks for each row to the executor, passing the collection explicitly
        futures = [executor.submit(extract_data_from_pdf, row, collection, pdf_op, OficioNombramientoAdministradores) for _, row in df.iterrows()]
        # Optionally, you can handle results or exceptions here
        for future in concurrent.futures.as_completed(futures):
            try:
                future.result()  # We can use this to raise exceptions if any occurred in the threads
            except Exception as e:
                # remove row from df if error
                print(f"Exception in thread: {e}")
                traceback.print_exc()


# Adjust the number of threads as needed
max_threads = 10  # Example: Use 5 threads

# Execute the function with multithreading, passing the collection explicitly
process_pdf_data_in_parallel(df_oficio_nombramiento_administradores, collection, max_threads)

Extracting data from 1391748027001_DocumentosGenerales_Oficio Nombramiento Administradores_2017-12-19_PALADINES LUISA MARLENE_GERENTE GENERAL.pdf
Extracting data from 1791338693001_DocumentosGenerales_Oficio Nombramiento Administradores_2020-12-04_VEINTIMILLA ALAVA LUIS GUILLERMO_APODERADO ESPECIAL.pdf
Extracting data from 0991037357001_DocumentosGenerales_Oficio Nombramiento Administradores_2021-07-08_LEON ASPIAZU JAVIER FRANCISCO_APODERADO.pdf
Extracting data from 0190485048001_DocumentosGenerales_Oficio Nombramiento Administradores_2019-12-05_VAZQUEZ CUEVA JORGE MAURICIO_PRESIDENTE.pdf
Extracting data from 0190485048001_DocumentosGenerales_Oficio Nombramiento Administradores_2019-12-05_VASQUEZ MALO JUAN FERNANDO_GERENTE GENERAL.pdf
Extracting data from 0993075345001_DocumentosGenerales_Oficio Nombramiento Administradores_2022-05-24_BETANCOURT ORTEGA DIANA LILIANA_APODERADO GENERAL.pdf
Extracting data from 1391794916001_DocumentosGenerales_Oficio Nombramiento Administradores_2017-10-

Traceback (most recent call last):
  File "/tmp/ipykernel_637815/2258620412.py", line 9, in process_pdf_data_in_parallel
    future.result()  # We can use this to raise exceptions if any occurred in the threads
    ^^^^^^^^^^^^^^^
  File "/usr/lib64/python3.12/concurrent/futures/_base.py", line 449, in result
    return self.__get_result()
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/lib64/python3.12/concurrent/futures/_base.py", line 401, in __get_result
    raise self._exception
  File "/usr/lib64/python3.12/concurrent/futures/thread.py", line 58, in run
    result = self.fn(*self.args, **self.kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_637815/2288641554.py", line 27, in extract_data_from_pdf
    tokens = len(text.split())
                 ^^^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'split'


Extracting data from 0990326606001_DocumentosGenerales_Oficio Nombramiento Administradores_2020-05-13_VITERI ANDRADE LEONARDO RENE_VICEPRESIDENTE EJECUTIVO.pdf
(1, 'Estimating resolution as 202 Warning. Invalid resolution 0 dpi. Using 70 instead. Too few characters. Skipping this page Error during processing.')
Exception in thread: 'NoneType' object has no attribute 'split'


Traceback (most recent call last):
  File "/tmp/ipykernel_637815/2258620412.py", line 9, in process_pdf_data_in_parallel
    future.result()  # We can use this to raise exceptions if any occurred in the threads
    ^^^^^^^^^^^^^^^
  File "/usr/lib64/python3.12/concurrent/futures/_base.py", line 449, in result
    return self.__get_result()
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/lib64/python3.12/concurrent/futures/_base.py", line 401, in __get_result
    raise self._exception
  File "/usr/lib64/python3.12/concurrent/futures/thread.py", line 58, in run
    result = self.fn(*self.args, **self.kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_637815/2288641554.py", line 27, in extract_data_from_pdf
    tokens = len(text.split())
                 ^^^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'split'


Extracting data from 1791251237001_DocumentosGenerales_Oficio Nombramiento Administradores_2018-12-04_MALDONADO GUEVARA FRANCISCO TEODORO_APODERADO GENERAL.pdf
(1, 'Estimating resolution as 113 Too few characters. Skipping this page Warning. Invalid resolution 0 dpi. Using 70 instead. Too few characters. Skipping this page Error during processing.')
Exception in thread: 'NoneType' object has no attribute 'split'
(1, 'Estimating resolution as 150 Too few characters. Skipping this page Warning. Invalid resolution 0 dpi. Using 70 instead. Too few characters. Skipping this page Error during processing.')
Exception in thread: 'NoneType' object has no attribute 'split'


...

(1, 'Warning. Invalid resolution 0 dpi. Using 70 instead. Too few characters. Skipping this page Error during processing.')
Exception in thread: 'NoneType' object has no attribute 'split'


Traceback (most recent call last):
  File "/tmp/ipykernel_637815/2258620412.py", line 9, in process_pdf_data_in_parallel
    future.result()  # We can use this to raise exceptions if any occurred in the threads
    ^^^^^^^^^^^^^^^
  File "/usr/lib64/python3.12/concurrent/futures/_base.py", line 449, in result
    return self.__get_result()
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/lib64/python3.12/concurrent/futures/_base.py", line 401, in __get_result
    raise self._exception
  File "/usr/lib64/python3.12/concurrent/futures/thread.py", line 58, in run
    result = self.fn(*self.args, **self.kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_637815/2288641554.py", line 27, in extract_data_from_pdf
    tokens = len(text.split())
                 ^^^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'split'


Extracting data from 0991307605001_DocumentosGenerales_Oficio Nombramiento Administradores_2022-09-28_CHOEZ MARQUEZ FERGIE DAYANA_APODERADO ESPECIAL DE LA SUCURSAL DE QUITO.pdfExtracting data from 0991307605001_DocumentosGenerales_Oficio Nombramiento Administradores_2020-02-13_ALMEIDA POLO DENISSE DEL ALBA_APODERADO ESPECIAL DE LA SUCURSAL DE QUITO.pdf

(1, 'Warning. Invalid resolution 0 dpi. Using 70 instead. Too few characters. Skipping this page Error during processing.')
Exception in thread: 'NoneType' object has no attribute 'split'


Traceback (most recent call last):
  File "/tmp/ipykernel_637815/2258620412.py", line 9, in process_pdf_data_in_parallel
    future.result()  # We can use this to raise exceptions if any occurred in the threads
    ^^^^^^^^^^^^^^^
  File "/usr/lib64/python3.12/concurrent/futures/_base.py", line 449, in result
    return self.__get_result()
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/lib64/python3.12/concurrent/futures/_base.py", line 401, in __get_result
    raise self._exception
  File "/usr/lib64/python3.12/concurrent/futures/thread.py", line 58, in run
    result = self.fn(*self.args, **self.kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_637815/2288641554.py", line 27, in extract_data_from_pdf
    tokens = len(text.split())
                 ^^^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'split'


Extracting data from 1790991334001_DocumentosGenerales_Oficio Nombramiento Administradores_2017-12-14_LOPEZ DONOSO HUGO FRANCISCO_APODERADO GENERAL.pdf
